✅\
Solution Proposée Pour l'**Atelier 1** et **Atelier 2**\
  Préparer par l'étudiant ***Ayoub BENHAIMOUD***

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin


# Vérification de l'environnement CUDA

In [ ]:
%%cu
#include<stdio.h>
__global__ void kernel(void) {}
int main (void)
{
kernel<<<10,1>>>();
printf(" H e l l o , World 2! \n") ;
//return 0;
}

 H e l l o , World 2! 



**Atelier 1 ✅**

Somme de deux réels simple précision.
1.	Ecrire le code C : exécution sur CPU
2.	Ecrire le code CUDA: exécution sur GPU \


✍\

Configuration:
-	1 seul thread qui effectue l’opération.
-	grille 1D de taille 1 blocs composé d’un seul thread



In [ ]:
%%cu
#include <stdio.h>


void add (float a, float b, float *c) {
      *c = a + b ;
}

int main ( void ) { 
    
    float a, b, c; 
    a=8.;
    b=25.;
    add (a, b, & c); 
    return 0;
}


In [ ]:
%%cu
#include<stdio.h>
__global__ void add (float *a, float *b, float *c) {
    *c = *a + *b ;
}
int main ( void ) {
    
    float a , b , c ;
    float *gpu_a , *gpu_b , *gpu_c ;
    int size = sizeof (float) ;
    // allocation de l’espace mémoire pour les variables gpu_a , gpu_b etg gpu_c
    // dans la mémoire globale du GPU
    cudaMalloc( (void **)&gpu_a, size) ;
    cudaMalloc( (void **)&gpu_b, size) ;
    cudaMalloc( (void **)&gpu_c, size) ;
    a=8. ;
    b=25. ;
    // Copie de des données (a et b) du Host ver le Device
    cudaMemcpy (gpu_a , &a, size , cudaMemcpyHostToDevice ) ;
    cudaMemcpy ( gpu_b , &b, size , cudaMemcpyHostToDevice ) ;
    // Configuration d’une grille de taille 1 bloc de taille 1 thread
    dim3 gridB; // par défaut dim3 initialise les coordonnés à 1
    dim3 blockT;
    add <<< gridB, blockT >>> ( gpu_a , gpu_b , gpu_c ) ;
    // ou tout simplement
    // add <<< 1 , 1 >>> ( gpu_a , gpu_b , gpu_c ) ;
    //Copie du résultat du GPU vers le Host
    cudaMemcpy (&c, gpu_c, size, cudaMemcpyDeviceToHost) ;
    printf("le resultat : %f",c );
    //Libération de l’espace alloué
    cudaFree(gpu_a) ;
    cudaFree( gpu_b) ;
    cudaFree ( gpu_c) ;
    return 0;
}





le resultat : 33.000000


**Atelier 2 ✅** \

Somme de deux vecteurs chacun de taille N\
 -Grille 1D de taille N/nbthreads blocs\
 -chaque bloc 1D de taille nbthreads

In [ ]:
%%cu
#include<stdio.h>
#include <iostream>
#include<stdlib.h>
#include<math.h>
#include <curand.h>
#include <curand_kernel.h>
#define N 2048 // taille des vecteurs N
#define nbthreads 64 // taille du bloc 64x64 (nb threads par block)
__global__ void add(float *a, float *b, float *c){
  int idx= blockIdx.x*blockDim.x+threadIdx.x;
  if (idx<N){
      // si nbthreads ne divise pas N
     *(c+idx)=*(a+idx)+*(b+idx);
     // c[idx]=a[idx]+b[idx];
     printf("le resultat : %f\n ", c[idx]);
  }
}

void random_ints(float* a, int size)
{
	int i,j;
	for (i=0;i<size;i++) {
		a[i]=rand()%100;
	}
}
int main(){
    
    float *a, *b, *c;
    float *gpu_a, *gpu_b, *gpu_c;
    int size = N*sizeof (float);
    // allocation de l’espace pour les vecteurs a, b et c sur le Host
    a=(float*) malloc(size);  random_ints(a, N);
    b=(float*) malloc(size);  random_ints(b, N);
    c=(float*) malloc(size);
    //printf("le resultat : %f ", c);
    // initialisation des vecteurs a et b.

    //random_floats(b, N);


    // allocation de l’espace mémoire dans la mémoire globale du device
    cudaMalloc ((void **)&gpu_a, size ) ;
    cudaMalloc ((void **)&gpu_b, size ) ;
    cudaMalloc ((void **)&gpu_c, size ) ;
    // Copie des données CPU vers GPU
    cudaMemcpy (gpu_a, a, size, cudaMemcpyHostToDevice );
    cudaMemcpy (gpu_b, b, size, cudaMemcpyHostToDevice );
    // Définition de la grille et appel du kernel
    // on définit la dimension des blocs 1D: nombre de threads par bloc
    dim3 dimB(nbthreads);
    // on définit la dimension de la grille 1D: nombre de blocs
    dim3 dimG(ceil(N/(float)nbthreads));
    // appel du kernel
    add<<<dimG,dimB>>>(gpu_a, gpu_b, gpu_c );
    // Copie du résultat du GPU vers CPU
    cudaMemcpy (c, gpu_c, size, cudaMemcpyDeviceToHost );

    free(a); free(b); free(c);
    cudaFree (gpu_a); cudaFree (gpu_b); cudaFree (gpu_c) ;
    return 0 ;
}

le resultat : 108.000000
 le resultat : 41.000000
 le resultat : 109.000000
 le resultat : 121.000000
 le resultat : 187.000000
 le resultat : 110.000000
 le resultat : 75.000000
 le resultat : 71.000000
 le resultat : 147.000000
 le resultat : 78.000000
 le resultat : 146.000000
 le resultat : 147.000000
 le resultat : 101.000000
 le resultat : 108.000000
 le resultat : 18.000000
 le resultat : 112.000000
 le resultat : 152.000000
 le resultat : 100.000000
 le resultat : 166.000000
 le resultat : 93.000000
 le resultat : 44.000000
 le resultat : 129.000000
 le resultat : 114.000000
 le resultat : 82.000000
 le resultat : 55.000000
 le resultat : 139.000000
 le resultat : 159.000000
 le resultat : 176.000000
 le resultat : 65.000000
 le resultat : 3.000000
 le resultat : 36.000000
 le resultat : 125.000000
 le resultat : 134.000000
 le resultat : 97.000000
 le resultat : 35.000000
 le resultat : 135.000000
 le resultat : 105.000000
 le resultat : 69.000000
 le resultat : 83.000000
 le 